In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation
# @User who created this repository: Joshua Zhang from Geog 458 AC section April 22, 2022

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets-2.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = ''
    consumer_secret = ""
    access_token = ""
    access_token_secret = ""
    # The tokens are hidden in this repository

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    LOCATIONS2 = [-25.835924951807126, 35.3761155416293,
                  29.535168417107514, 70.78168810314003] # West European countries

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS2, languages=['en'], encoding="utf-8")

{'id': 1517629829524434946, 'username': 'Lee_Bickerton', 'created_at': 'Fri Apr 22 22:22:02 +0000 2022', 'lng': -2.373384, 'lat': 52.6667175, 'text': 'That’s the set! Huge thanks Peter L. A thoughtful gift! - Drinking a Nuclear Pasta by @VariantBrewing @ Shifnal  — https://t.co/Nb6bMbAn06'}
{'id': 1517629832565252098, 'username': 'ARcoaching91', 'created_at': 'Fri Apr 22 22:22:02 +0000 2022', 'lng': -1.172266, 'lat': 52.969719, 'text': '@HLMustangs1 Mental mate, lads gave it the whole "strong finish to the season" speech and most of the people away a… https://t.co/J2rLNlh7TO'}
{'id': 1517629834377183232, 'username': 'Ahmed__Abukur', 'created_at': 'Fri Apr 22 22:22:03 +0000 2022', 'lng': -1.8906405, 'lat': 52.4939665, 'text': '@MuhammadKummo Amin🤲'}
{'id': 1517629834515648516, 'username': 'Dooftronic', 'created_at': 'Fri Apr 22 22:22:03 +0000 2022', 'lng': -8.979993, 'lat': 38.8582775, 'text': 'Fucking glorious!'}
{'id': 1517629837753843713, 'username': 'BannerGAAEnnis', 'created_at': '

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
